In [21]:
!pip install --upgrade dash

!pip install dash dash-table

!pip install dash_bootstrap_components

!pip install numpy-financial

!pip install dash_auth


In [35]:

import pandas as pd
# Read the first few lines of the CSV
with open('SUN Hour.csv', 'r') as file:
    for i, line in enumerate(file):
        if i < 10:  # Print first 10 lines
            print(f"Line {i}: {line.strip()}")
# Load without skipping rows to check raw data
weather_df_raw = pd.read_csv('SUN Hour.csv', sep=';', header=None)
print("Raw columns:", weather_df_raw.shape[1])
print("Raw first 5 rows:")
print(weather_df_raw.head())
# Load with current settings to check TmStamp values
weather_df = pd.read_csv('SUN Hour.csv', sep=';', skiprows=3, header=None)
weather_df = weather_df.iloc[:, :5]
weather_df.columns = ['TmStamp', 'SunWM_Avg', 'AirTC_Avg', 'RH', 'WS_ms_S_WVT']
print("TmStamp sample values:", weather_df['TmStamp'].head(10).tolist())



Line 0: ï»¿SUN - Stellenbosch University;Latitude: -33.92810059;Longitude: 18.86540031;Elevation: 119 m;;;;;;;;;;;;;;;
Line 1: TmStamp;RecNum;BattV_Min;TrackerWM_Avg;Tracker2WM_Avg;ShadowWM_Avg;SunWM_Avg;ShadowbandWM_Avg;DNICalc_Avg;AirTC_Avg;RH;WS_ms_S_WVT;WindDir_D1_WVT;WindDir_SD1_WVT;BP_mB_Avg;UVA_Avg;UVB_Avg;Batt24V_Min;
Line 2: ;;Volts;kW/m^2;;kW/m^2;W/m^2;W/m^2;;Deg C;%;meters/second;Deg;Deg;;W/m;W/m;;
Line 3: TmStamp;RecNum;Min;Avg;Avg;Avg;Avg;Avg;Avg;Avg;Smp;WVc;WVc;WVc;Avg;Avg;Avg;;
Line 4: 01/01/2023 00:00:00;2527;13.66;0;0;0;0;0;0;23.15;62.09;1.164;182;38.66;999.03723;0.0010540753;0.006824831;26.27;
Line 5: 01/01/2023 01:00:00;2528;13.67;0;0;0;0;0;0;21.52;67.01;1.631;170.6;35.59;998.58435;0.00323097;0.006917045;26.26;
Line 6: 01/01/2023 02:00:00;2529;13.64;0;0;0;0;0;0;20.91;69.96;1.445;158.4;34.9;998.1258;0.007080636;0.006853746;26.26;
Line 7: 01/01/2023 03:00:00;2530;13.54;0;0;0;0;0;0;19.43;74.56;1.417;155.7;30.49;997.5622;0.01369725;0.00734412;26.26;
Line 8: 01/01/2023 04

In [31]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import numpy_financial as npf
from dash import Dash, dcc, html, Input, Output, State, callback_context
import dash_bootstrap_components as dbc
import dash_auth
import warnings
from datetime import datetime
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import base64

warnings.filterwarnings('ignore')

# Data Preparation
def load_and_preprocess_data():
    try:
        load_df = pd.read_csv('Industrial-Plastic Manufacturer.csv', sep=';')
        weather_df = pd.read_csv('SUN Hour.csv', sep=';', skiprows=1)
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Data file not found: {e}")

    load_df['Power_kW'] = pd.to_numeric(load_df['Power_kW'], errors='coerce')
    load_df['Power_kW'] = load_df['Power_kW'].interpolate(method='linear', limit_direction='both')
    mask = (load_df['Power_kW'] == 0) & (load_df.index > 0) & (load_df.index < len(load_df)-1)
    load_df.loc[mask, 'Power_kW'] = np.nan
    load_df['Power_kW'] = load_df['Power_kW'].interpolate(method='linear')
    load_df['Power_kW'] = load_df['Power_kW'].clip(lower=0)
    
    weather_df['TmStamp'] = pd.to_datetime(weather_df['TmStamp'], errors='coerce', format='%d/%m/%Y %H:%M:%S')
    weather_df = weather_df.dropna(subset=['TmStamp'])
    weather_df['Hour'] = weather_df['TmStamp'].dt.hour
    weather_df['Month'] = weather_df['TmStamp'].dt.month
    weather_df['DayOfWeek'] = weather_df['TmStamp'].dt.dayofweek
    numeric_cols = ['SunWM_Avg', 'AirTC_Avg', 'RH', 'WS_ms_S_WVT']
    weather_df[numeric_cols] = weather_df[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(method='ffill')
    weather_df.loc[(weather_df['Hour'] >= 18) | (weather_df['Hour'] < 6), 'SunWM_Avg'] = 0

    load_df['DateTime'] = pd.date_range(start=weather_df['TmStamp'].min(), 
                                        periods=len(load_df), freq='H')
    load_df['DateTime'] = load_df['DateTime'].dt.tz_localize(None).dt.floor('H')
    weather_df['TmStamp'] = weather_df['TmStamp'].dt.tz_localize(None).dt.floor('H')
    merged_df = pd.merge(load_df, weather_df, left_on='DateTime', right_on='TmStamp', how='inner')
    
    if merged_df.empty:
        raise ValueError("Merged DataFrame is empty. Check timestamp alignment.")

    merged_df = merged_df.rename(columns={'DateTime': 'Timestamp'})
    
    max_demand = merged_df['Power_kW'].max()
    print(f"Maximum Power_kW after cleaning: {max_demand:.2f} kW")
    zero_count = (merged_df['Power_kW'] == 0).sum()
    print(f"Number of zero values in Power_kW: {zero_count}")

    return merged_df

# System Sizing
def size_hybrid_system(load_offset_percent, pv_percent, gen_percent, battery_daily_energy_percent, 
                       annual_energy_kwh, max_demand_kw, peak_sun_hours, pv_system_efficiency, dod, gen_safety_factor):
    total_percent = pv_percent + gen_percent
    if abs(total_percent - 100) > 0.01:
        return None, "PV and GEN percentages must sum to 100%"
    if any(x < 0 for x in [load_offset_percent, pv_percent, gen_percent, battery_daily_energy_percent, 
                           peak_sun_hours, pv_system_efficiency, dod, gen_safety_factor]):
        return None, "Invalid input values (negative values not allowed)"
    if any(x <= 0 for x in [load_offset_percent, peak_sun_hours, pv_system_efficiency, dod, gen_safety_factor]):
        return None, "Invalid input values (zero or negative values not allowed)"

    energy_to_cover_kwh = (load_offset_percent / 100) * annual_energy_kwh
    daily_energy_kwh = energy_to_cover_kwh / 365

    if battery_daily_energy_percent == 0:
        battery_daily_energy_kwh = 0
        battery_capacity_kwh = 0
        battery_capacity_mwh = 0
        battery_inverter_kw = 0
        battery_cost = 0
    else:
        battery_daily_energy_kwh = (battery_daily_energy_percent / 100) * daily_energy_kwh
        battery_capacity_kwh = battery_daily_energy_kwh / dod
        battery_capacity_mwh = battery_capacity_kwh / 1000
        battery_inverter_kw = max_demand_kw
        battery_cost = battery_capacity_kwh * 400

    pv_load_kwh = energy_to_cover_kwh * (pv_percent / 100)
    pv_battery_kwh = battery_daily_energy_kwh * 365
    pv_energy_kwh = pv_load_kwh + pv_battery_kwh
    pv_capacity_kwp = pv_energy_kwh / (peak_sun_hours * 365 * pv_system_efficiency)
    inverter_capacity_kw = pv_capacity_kwp / 1.2
    num_panels = int(np.ceil(pv_capacity_kwp * 1000 / 600))
    pv_cost = pv_capacity_kwp * 1000

    gen_energy_kwh = energy_to_cover_kwh - pv_load_kwh
    gen_capacity_kw = max_demand_kw * gen_safety_factor
    gen_capacity_kw = round(gen_capacity_kw / 100) * 100
    gen_capacity_kwh = gen_energy_kwh / 365
    gen_cost = gen_capacity_kw * 500

    total_cost = pv_cost + battery_cost + gen_cost

    return {
        'energy_to_cover_kwh': energy_to_cover_kwh,
        'pv_capacity_kwp': pv_capacity_kwp,
        'inverter_capacity_kw': inverter_capacity_kw,
        'pv_energy_kwh': pv_energy_kwh,
        'pv_num_panels': num_panels,
        'pv_cost': pv_cost,
        'battery_capacity_mwh': battery_capacity_mwh,
        'battery_inverter_kw': battery_inverter_kw,
        'battery_daily_energy_kwh': battery_daily_energy_kwh,
        'battery_cost': battery_cost,
        'gen_capacity_kw': gen_capacity_kw,
        'gen_capacity_kwh': gen_capacity_kwh,
        'gen_energy_kwh': gen_energy_kwh,
        'gen_cost': gen_cost,
        'total_cost': total_cost,
        'load_offset_percent': load_offset_percent,
        'pv_percent': pv_percent,
        'gen_percent': gen_percent,
        'battery_daily_energy_percent': battery_daily_energy_percent,
        'peak_sun_hours': peak_sun_hours,
        'pv_system_efficiency': pv_system_efficiency,
        'dod': dod,
        'gen_safety_factor': gen_safety_factor
    }, ""

# Energy Source Assignment
def assign_energy_sources(pv_capacity_kwp, df, system_efficiency=0.85, has_battery=True):
    df = df.copy()
    df['Temp_Derating'] = 1 - 0.004 * (df['AirTC_Avg'] - 25).clip(lower=0)
    df['PV_Output_kW'] = pv_capacity_kwp * (df['SunWM_Avg'] / 1000) * system_efficiency * df['Temp_Derating']
    df['Source'] = 'GEN'
    df['Source_kWh'] = df['Power_kW'] / 2
    df.loc[(df['Hour'] >= 6) & (df['Hour'] < 18) & (df['PV_Output_kW'] >= df['Power_kW'] * 0.5), 'Source'] = 'PV'
    df.loc[(df['Hour'] >= 6) & (df['Hour'] < 18) & (df['PV_Output_kW'] >= df['Power_kW'] * 0.5), 'Source_kWh'] = df['PV_Output_kW'] / 2
    if has_battery:
        df.loc[(df['Hour'] >= 18) | (df['Hour'] < 6) & (df['Power_kW'] > 0), 'Source'] = 'BESS'
        df.loc[(df['Hour'] >= 18) | (df['Hour'] < 6) & (df['Power_kW'] > 0), 'Source_kWh'] = df['Power_kW'] / 2
    else:
        df.loc[(df['Source'] != 'PV') & (df['Power_kW'] > 0), 'Source'] = 'GEN'
        df.loc[(df['Source'] != 'PV') & (df['Power_kW'] > 0), 'Source_kWh'] = df['Power_kW'] / 2
    return df

# Financial Calculations (ZAR)
def calculate_financial_metrics(sizing_data, pv_cost_per_kwp, bess_cost_per_kwh, gen_cost_per_kw, 
                               discount_rate, project_lifetime, electricity_cost, 
                               pv_om_percent, bess_om_percent, gen_om_percent, fuel_type, fuel_cost):
    if not sizing_data:
        return None, "No sizing data available"

    pv_capacity_kwp = sizing_data['pv_capacity_kwp']
    battery_capacity_kwh = sizing_data['battery_capacity_mwh'] * 1000
    gen_capacity_kw = sizing_data['gen_capacity_kw']
    pv_energy_kwh = sizing_data['pv_energy_kwh']
    gen_energy_kwh = sizing_data['gen_energy_kwh']
    total_energy_kwh = sizing_data['energy_to_cover_kwh']

    pv_cost = pv_capacity_kwp * pv_cost_per_kwp
    bess_cost = battery_capacity_kwh * bess_cost_per_kwh
    gen_cost = gen_capacity_kw * gen_cost_per_kw
    total_capital_cost = pv_cost + bess_cost + gen_cost

    pv_om_cost = pv_cost * pv_om_percent / 100
    bess_om_cost = bess_cost * bess_om_percent / 100
    gen_om_cost = gen_cost * gen_om_percent / 100
    total_om_cost = pv_om_cost + bess_om_cost + gen_om_cost

    fuel_consumption_rates = {
        'Diesel': 0.25,
        'Compressed Biogas': 0.3,
        'Natural Gas': 0.3,
        'Gasoline': 0.3
    }
    fuel_consumption = gen_energy_kwh * fuel_consumption_rates[fuel_type]
    annual_fuel_cost = fuel_consumption * fuel_cost

    annual_savings = total_energy_kwh * electricity_cost

    cash_flows = [-total_capital_cost]
    annual_net_cash_flow = annual_savings - total_om_cost - annual_fuel_cost
    for _ in range(project_lifetime):
        cash_flows.append(annual_net_cash_flow)

    npv = npf.npv(discount_rate / 100, cash_flows)
    try:
        irr = npf.irr(cash_flows) * 100 if cash_flows else 0
    except:
        irr = 0

    cumulative_cash_flow = 0
    payback_period = 0
    for year, cf in enumerate(cash_flows[1:], 1):
        cumulative_cash_flow += cf
        if cumulative_cash_flow >= total_capital_cost:
            payback_period = year - 1 + (total_capital_cost - (cumulative_cash_flow - cf)) / cf
            break
    if payback_period == 0 and cumulative_cash_flow < total_capital_cost:
        payback_period = float('inf')

    total_costs = total_capital_cost + sum([(total_om_cost + annual_fuel_cost) / ((1 + discount_rate / 100) ** t) for t in range(1, project_lifetime + 1)])
    total_energy = total_energy_kwh * project_lifetime
    lcoe = total_costs / total_energy if total_energy > 0 else float('inf')

    grid_emission_factor = 0.7
    co2_savings_pv = pv_energy_kwh * grid_emission_factor

    fuel_emission_factors = {
        'Diesel': 2.68 * 0.25,
        'Compressed Biogas': 0.45,
        'Natural Gas': 0.4,
        'Gasoline': 2.31 * 0.3
    }
    co2_emissions_gen = gen_energy_kwh * fuel_emission_factors[fuel_type]

    trees_planted = co2_savings_pv / 25

    return {
        'total_capital_cost': total_capital_cost,
        'pv_cost': pv_cost,
        'bess_cost': bess_cost,
        'gen_cost': gen_cost,
        'npv': npv,
        'irr': irr,
        'payback_period': payback_period,
        'lcoe': lcoe,
        'co2_savings_pv': co2_savings_pv,
        'co2_emissions_gen': co2_emissions_gen,
        'trees_planted': trees_planted,
        'annual_savings': annual_savings,
        'total_om_cost': total_om_cost,
        'annual_fuel_cost': annual_fuel_cost,
        'cash_flows': cash_flows
    }

# Predictive Model
def train_predict_pv_load(df, pv_capacity_kwp, start_date, end_date):
    df = df.copy()
    df['Temp_Derating'] = 1 - 0.004 * (df['AirTC_Avg'] - 25).clip(lower=0)
    df['PV_Output_kW'] = pv_capacity_kwp * (df['SunWM_Avg'] / 1000) * 0.85 * df['Temp_Derating']
    
    features = ['Hour', 'DayOfWeek', 'Month', 'SunWM_Avg', 'AirTC_Avg']
    targets = ['PV_Output_kW', 'Power_kW']
    X = df[features]
    
    models = {}
    mse = {}
    for target in targets:
        y = df[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
        model = XGBRegressor(n_estimators=100, learning_rate=0.1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse[target] = mean_squared_error(y_test, y_pred)
        models[target] = model
    
    period_df = df[(df['Timestamp'] >= start_date) & (df['Timestamp'] <= end_date)]
    if period_df.empty:
        return None, None, None, f"No data available for {start_date} to {end_date}"
    
    X_period = period_df[features]
    pred_pv = models['PV_Output_kW'].predict(X_period)
    pred_load = models['Power_kW'].predict(X_period)
    
    print(f"PV_Output_kW MSE: {mse['PV_Output_kW']:.2f}")
    print(f"Power_kW MSE: {mse['Power_kW']:.2f}")
    
    return pred_pv, pred_load, period_df['Timestamp'], None

# Initialize Dash app with authentication
VALID_USERNAME_PASSWORD_PAIRS = {
    'professor': 'Power874',
    'student': 'kapfunde27692566'
}

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)
auth = dash_auth.BasicAuth(app, VALID_USERNAME_PASSWORD_PAIRS)
app.layout = dbc.Container([
    html.H1("Hybrid Energy System Dashboard", className="text-center my-4"),
    dcc.Tabs(id="tabs", value='project', children=[
        dcc.Tab(label='Project Information', value='project'),
        dcc.Tab(label='Weather Data Visualization', value='weather'),
        dcc.Tab(label='System Sizing', value='sizing'),
        dcc.Tab(label='Energy Usage', value='energy'),
        dcc.Tab(label='Financials', value='financials'),
        dcc.Tab(label='Predictions', value='predictions')
    ]),
    html.Div(id='tabs-content'),
    dcc.Store(id='sizing-store'),
    html.Div([
        html.Button("Download Dashboard as PDF", id='download-pdf-btn', n_clicks=0, 
                    className="btn btn-primary mt-3", style={'display': 'block', 'margin': 'auto'}),
        dcc.Download(id='download-pdf')
    ], className="text-center")
], fluid=True)

# Load data
try:
    df = load_and_preprocess_data()
    annual_energy_kwh = df['Power_kW'].sum() / 2
    max_demand_kw = df['Power_kW'].max()
except Exception as e:
    print(f"Error loading data: {e}")
    exit(1)

# Color palette
colors = {
    'primary': '#00BFFF',
    'secondary': '#FF4500',
    'accent': '#32CD32',
    'background': '#f5f5f5',
    'text': '#333333',
    'load': '#000000',
    'none': '#D3D3D3'
}

# Week options for 2023
week_options = []
start_date = pd.to_datetime('2023-01-01')
for i in range(52):
    week_start = start_date + pd.Timedelta(days=i*7)
    week_end = week_start + pd.Timedelta(days=6)
    week_label = f"Week {i+1}: {week_start.strftime('%b %d')}–{week_end.strftime('%b %d')}"
    week_options.append({'label': week_label, 'value': i})

# Callbacks
@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value')
)
def render_content(tab):
    if tab == 'project':
        return html.Div([
            html.Img(src="assets/Logo.jpg", 
                     style={'height': '150px', 'display': 'block', 'margin': '20px auto', 'border-radius': '10px'}),
            html.H1("Stellenbosch University", className="text-center", style={'font-family': 'Arial', 'color': colors['text']}),
            html.H3("Faculty of Engineering", className="text-center", style={'font-family': 'Arial', 'color': colors['text']}),
            html.H3("Department of Electrical & Electronics Engineering", className="text-center", 
                    style={'font-family': 'Arial', 'color': colors['text']}),
            html.H4("Power System Data Analytics 874 (Final Project)", className="text-center mt-4", 
                    style={'font-family': 'Arial', 'color': colors['text']}),
            html.P("This dashboard facilitates the design and analysis of off-grid hybrid energy systems. It enables users to size photovoltaic (PV), battery, and generator components to meet a specified load offset, visualize energy usage patterns through interactive heatmaps and charts, and explore weather data impacting system performance. Developed for industrial applications, the tool provides dynamic, data-driven insights for sustainable energy planning.", 
                   className="text-center mt-4", style={'font-family': 'Arial', 'font-size': '16px', 'font-style': 'italic', 'color': colors['text']}),
            html.H5("Panashe Bradley Kapfunde (27692566)", className="text-center mt-4", 
                    style={'font-family': 'Arial', 'color': colors['text']}),
            html.H5("Title: A Data Visualization Dashboard for Sizing Battery and PV Systems and Predicting Energy Output and Savings for Off-Grid Energy Systems", 
                    className="text-center mt-4", style={'font-family': 'Arial', 'color': colors['text'], 'font-style': 'italic'}),
            html.Div([
                html.A("Centre for Renewable and Sustainable Energy Studies (CRESE)", 
                       href="http://www.crses.sun.ac.za/", target="_blank",
                       className="btn btn-primary mx-2 mt-5"),
                html.A("Department of Electrical and Electronic Engineering", 
                       href="http://www.ee.sun.ac.za/", target="_blank", 
                       className="btn btn-primary mx-2 mt-5")
            ], className="text-center")
        ], style={'background-color': '#ffffff', 'padding': '40px', 'border-radius': '10px', 'margin': '20px auto', 'max-width': '800px'})
    
    elif tab == 'weather':
        weather_options = [
            {'label': 'Irradiance (W/m²)', 'value': 'SunWM_Avg'},
            {'label': 'Temperature (°C)', 'value': 'AirTC_Avg'},
            {'label': 'Humidity (%)', 'value': 'RH'},
            {'label': 'Wind Speed (m/s)', 'value': 'WS_ms_S_WVT'}
        ]
        return html.Div([
            html.H3("Weather Data Visualization", className="text-center mb-4"),
            dbc.Row([
                dbc.Col([
                    html.Label("Select Weather Variable", className="font-weight-bold"),
                    dcc.Dropdown(id='weather-variable', options=weather_options, value='SunWM_Avg', className="mb-3"),
                    dcc.Graph(id='time-series', style={'height': '600px'}),
                    html.P("This time series plot shows hourly values of the selected weather variable over 2023.", 
                           className="text-muted mt-2")
                ], md=6),
                dbc.Col([
                    dcc.Graph(id='box-plot', style={'height': '600px'}),
                    html.P("This box plot illustrates the distribution of the selected weather variable by month in 2023.", 
                           className="text-muted mt-2")
                ], md=6)
            ], className="mb-4"),
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id='temp-heatmap', style={'height': '600px'}),
                    html.P("This heatmap shows hourly temperature (°C) across days of the year in 2023.", 
                           className="text-muted mt-2")
                ], md=6),
                dbc.Col([
                    dcc.Graph(id='irr-heatmap', style={'height': '600px'}),
                    html.P("This heatmap shows hourly irradiance (W/m²) across days of the year in 2023.", 
                           className="text-muted mt-2")
                ], md=6)
            ], className="mb-4"),
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id='scatter-plot', style={'height': '600px'}),
                    html.P("This scatter plot shows the relationship between temperature and irradiance in 2023.", 
                           className="text-muted mt-2")
                ], md=6),
                dbc.Col([
                    dcc.Graph(id='histogram', style={'height': '600px'}),
                    html.P("This histogram shows the distribution of the selected weather variable in 2023.", 
                           className="text-muted mt-2")
                ], md=6)
            ], className="mb-4"),
            dbc.Row([
                dbc.Col([
                    html.Button("Download Weather Data", id='download-weather-btn', 
                                n_clicks=0, className="btn btn-primary mb-2 mr-2"),
                    html.Button("Download Load Data", id='download-load-btn', 
                                n_clicks=0, className="btn btn-primary mb-2"),
                    dcc.Download(id='download-weather'),
                    dcc.Download(id='download-load')
                ], md=6, className="d-flex align-items-center justify-content-center")
            ]),
            html.Div(id='download-message', className="mt-2 text-center")
        ], style={'background-color': colors['background'], 'padding': '20px'})
    
    elif tab == 'sizing':
        return html.Div([
            html.H3("System Sizing", className="text-center"),
            dbc.Row([
                dbc.Col([
                    html.Label("Load Offset (%)"),
                    dcc.Slider(id='load-offset-slider', min=10, max=100, step=5, value=50, 
                               marks={10: '10%', 50: '50%', 100: '100%'}),
                    html.Div(id='load-offset-value', className="mb-3 text-muted"),
                    html.Label("PV Percentage (%)"),
                    dcc.Slider(id='pv-percent-slider', min=0, max=100, step=5, value=80, 
                               marks={0: '0%', 50: '50%', 100: '100%'}),
                    html.Div(id='pv-percent-value', className="mb-3 text-muted"),
                    html.Label("GEN Percentage (%)"),
                    dcc.Slider(id='gen-percent-slider', min=0, max=100, step=5, value=20, 
                               marks={0: '0%', 50: '50%', 100: '100%'}),
                    html.Div(id='gen-percent-value', className="mb-3 text-muted"),
                    html.Label("BESS Daily Energy (%)"),
                    dcc.Slider(id='battery-daily-energy-slider', min=0, max=100, step=5, value=50, 
                               marks={0: '0%', 50: '50%', 100: '100%'}),
                    html.Div(id='battery-daily-energy-value', className="mb-3 text-muted"),
                    html.Label("Peak Sun Hours"),
                    dcc.Input(id='peak-sun-hours', type='number', min=3, max=7, step=0.1, value=5, 
                              className="form-control mb-3"),
                    html.Label("PV System Efficiency"),
                    dcc.Dropdown(id='pv-system-efficiency', 
                                 options=[{'label': f'{x*100:.0f}%', 'value': x} for x in [0.80, 0.85, 0.90]], 
                                 value=0.85, className="mb-3"),
                    html.Label("BESS Depth of Discharge"),
                    dcc.Dropdown(id='dod', 
                                 options=[{'label': f'{x*100:.0f}%', 'value': x} for x in [0.70, 0.80, 0.90]], 
                                 value=0.80, className="mb-3"),
                    html.Label("GEN Safety Factor"),
                    dcc.Input(id='gen-safety-factor', type='number', min=1.0, max=1.5, step=0.1, value=1.2, 
                              className="form-control mb-3"),
                    html.Button("Reset Sliders", id='reset-button', n_clicks=0, className="btn btn-secondary mt-3")
                ], md=4),
                dbc.Col([
                    html.H4("Sizing Results", className="text-center"),
                    html.Div(id='sizing-results', className="mt-4"),
                    html.Button("Download Sizing Results", id='download-sizing-btn', 
                                n_clicks=0, className="btn btn-primary mt-2"),
                    html.Div(id='download-sizing-message', className="mt-2"),
                    dcc.Download(id='download-sizing')
                ], md=8)
            ]),
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id='energy-mix-pie', style={'height': '800px'}),
                    html.P("This pie chart shows the annual energy contribution from PV and GEN for 2023.", 
                           className="text-muted mt-2")
                ], md=12)
            ], className="mt-4")
        ])
    
    elif tab == 'energy':
        return html.Div([
            html.H3("Energy Usage for Selected System Configuration", className="text-center"),
            dbc.Row([
                dbc.Col([
                    html.Label("Select Week in 2023", style={'font-size': '18px'}),
                    dcc.Dropdown(id='week-selector', options=week_options, value=0, style={'width': '50%'}),
                    html.Button("Reset to Week 1", id='reset-week-button', n_clicks=0, 
                                className="btn btn-secondary mt-2 mb-3"),
                    dcc.Graph(id='weekly-source-heatmap', style={'height': '1000px'}),
                    html.P("This heatmap shows the dominant energy source by hour and day for the selected week in 2023.", 
                           className="text-muted mt-2"),
                    dcc.Graph(id='monthly-source-heatmap', style={'height': '1000px'}),
                    html.P("This heatmap shows the dominant energy source by hour and month for 2023.", 
                           className="text-muted mt-2"),
                    dcc.Graph(id='stacked-area', style={'height': '800px'}),
                    html.P("This stacked area chart shows the hourly energy contribution from PV, BESS, and GEN for the selected week.", 
                           className="text-muted mt-2"),
                    dcc.Graph(id='source-pie', style={'height': '800px'}),
                    html.P("This pie chart shows the energy supplied by PV, BESS, and GEN for the selected week.", 
                           className="text-muted mt-2"),
                    html.H4("Weekly Energy Summary", className="text-center mt-4"),
                    html.Table(id='energy-summary-table', className="table table-bordered table-striped", 
                               style={'font-size': '16px', 'margin': 'auto', 'width': '50%'}),
                ], md=12)
            ])
        ])
    
    elif tab == 'financials':
        return html.Div([
            html.H3("Financial Analysis and CO2 Impact", className="text-center mb-4"),
            dbc.Row([
                dbc.Col([
                    html.Label("PV Cost (R/kWp)", className="font-weight-bold"),
                    dcc.Input(id='pv-cost-input', type='number', min=500, max=2000, step=50, value=1000, 
                              className="form-control mb-3"),
                    html.Label("BESS Cost (R/kWh)", className="font-weight-bold"),
                    dcc.Input(id='bess-cost-input', type='number', min=200, max=600, step=50, value=400, 
                              className="form-control mb-3"),
                    html.Label("Generator Cost (R/kW)", className="font-weight-bold"),
                    dcc.Input(id='gen-cost-input', type='number', min=300, max=1000, step=50, value=500, 
                              className="form-control mb-3"),
                    html.Label("Generator Fuel Type", className="font-weight-bold"),
                    dcc.Dropdown(id='fuel-type', 
                                 options=[
                                     {'label': 'Diesel', 'value': 'Diesel'},
                                     {'label': 'Compressed Biogas', 'value': 'Compressed Biogas'},
                                     {'label': 'Natural Gas', 'value': 'Natural Gas'},
                                     {'label': 'Gasoline', 'value': 'Gasoline'}
                                 ], 
                                 value='Diesel', className="mb-3"),
                    html.Label("Fuel Cost (R/unit)", className="font-weight-bold"),
                    dcc.Input(id='fuel-cost-input', type='number', min=0.1, max=2.0, step=0.1, value=1.2, 
                              className="form-control mb-3"),
                    html.Label("Discount Rate (%)", className="font-weight-bold"),
                    dcc.Input(id='discount-rate', type='number', min=0, max=10, step=0.5, value=5, 
                              className="form-control mb-3"),
                    html.Label("Project Lifetime (years)", className="font-weight-bold"),
                    dcc.Input(id='project-lifetime', type='number', min=10, max=30, step=1, value=25, 
                              className="form-control mb-3"),
                    html.Label("Electricity Cost (R/kWh)", className="font-weight-bold"),
                    dcc.Input(id='electricity-cost', type='number', min=0.05, max=0.5, step=0.01, value=0.15, 
                              className="form-control mb-3"),
                    html.Label("PV O&M Cost (% of capital)", className="font-weight-bold"),
                    dcc.Input(id='pv-om', type='number', min=0, max=5, step=0.1, value=1, 
                              className="form-control mb-3"),
                    html.Label("BESS O&M Cost (% of capital)", className="font-weight-bold"),
                    dcc.Input(id='bess-om', type='number', min=0, max=5, step=0.1, value=2, 
                              className="form-control mb-3"),
                    html.Label("Generator O&M Cost (% of capital)", className="font-weight-bold"),
                    dcc.Input(id='gen-om', type='number', min=0, max=5, step=0.1, value=3, 
                              className="form-control mb-3"),
                ], md=4, className="p-3 bg-light rounded shadow-sm"),
                dbc.Col([
                    html.H4("Financial Metrics", className="text-center mb-4"),
                    dbc.Card([
                        dbc.CardBody([
                            html.H5("Key Financial Metrics", className="card-title text-center"),
                            html.Div(id='financial-metrics-table', className="mt-3")
                        ])
                    ], className="mb-4 shadow"),
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='cost-breakdown-pie', style={'height': '500px'})
                        ])
                    ], className="mb-4 shadow"),
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='payback-period-chart', style={'height': '500px'})
                        ])
                    ], className="mb-4 shadow"),
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='lcoe-gauge', style={'height': '400px'})
                        ])
                    ], className="mb-4 shadow"),
                    html.H4("Environmental Impact", className="text-center mb-4"),
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='co2-impact-chart', style={'height': '500px'}),
                            html.P(id='co2-savings-text', className="text-center mt-3", style={'font-size': '16px'})
                        ])
                    ], className="mb-4 shadow"),
                ], md=8, className="d-flex flex-column align-items-center")
            ], className="justify-content-center")
        ], style={'background-color': colors['background'], 'padding': '20px'})
    
    elif tab == 'predictions':
        return html.Div([
            html.H3("PV and Load Predictions", className="text-center mb-4", 
                    style={'font-family': 'Arial', 'font-weight': 'bold', 'font-size': '28px', 'color': colors['text']}),
            html.P("Select a week in 2023 to view predicted PV generation and load demand based on historical weather and load data.", 
                   className="text-center mb-4", style={'font-family': 'Arial', 'font-size': '16px', 'color': colors['text']}),
            dbc.Card([
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col([
                            html.Label("Select Week in 2023", className="font-weight-bold", 
                                       style={'font-family': 'Arial', 'font-size': '18px', 'color': colors['text']}),
                            dcc.Dropdown(
                                id='pred-week-selector', 
                                options=week_options, 
                                value=0, 
                                className="mb-3", 
                                style={'width': '100%', 'font-family': 'Arial', 'font-size': '16px'}
                            )
                        ], md=6, className="mx-auto")
                    ], className="mb-4"),
                    dcc.Graph(id='prediction-plot', style={'height': '700px'}),
                    html.P(id='prediction-message', className="mt-3 text-center", 
                           style={'font-family': 'Arial', 'font-size': '16px', 'color': colors['accent']})
                ])
            ], className="shadow", style={'background-color': '#ffffff', 'border-radius': '10px', 'padding': '20px'})
        ], style={'background-color': colors['background'], 'padding': '20px'})

@app.callback(
    [Output('pv-percent-slider', 'value'),
     Output('gen-percent-slider', 'value')],
    [Input('pv-percent-slider', 'value'),
     Input('gen-percent-slider', 'value'),
     Input('reset-button', 'n_clicks')],
    prevent_initial_call=True
)
def update_sliders(pv_value, gen_value, reset_n_clicks):
    ctx = callback_context
    if not ctx.triggered:
        return dash.no_update

    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if trigger_id == 'reset-button' and reset_n_clicks > 0:
        return 80, 20
    
    total = 100
    if trigger_id == 'pv-percent-slider' and pv_value is not None:
        gen_value = max(0, total - pv_value)
        return pv_value, gen_value
    elif trigger_id == 'gen-percent-slider' and gen_value is not None:
        pv_value = max(0, total - gen_value)
        return pv_value, gen_value
    
    return dash.no_update

@app.callback(
    [Output('load-offset-value', 'children'),
     Output('pv-percent-value', 'children'),
     Output('gen-percent-value', 'children'),
     Output('battery-daily-energy-value', 'children')],
    [Input('load-offset-slider', 'value'),
     Input('pv-percent-slider', 'value'),
     Input('gen-percent-slider', 'value'),
     Input('battery-daily-energy-slider', 'value')]
)
def update_slider_values(load_offset, pv_percent, gen_percent, battery_daily_energy_percent):
    return (f"Current: {load_offset}%", f"Current: {pv_percent}%", 
            f"Current: {gen_percent}%", f"Current: {battery_daily_energy_percent}%")

@app.callback(
    [Output('time-series', 'figure'),
     Output('box-plot', 'figure'),
     Output('temp-heatmap', 'figure'),
     Output('irr-heatmap', 'figure'),
     Output('scatter-plot', 'figure'),
     Output('histogram', 'figure')],
    Input('weather-variable', 'value')
)
def update_weather_plots(variable):
    labels = {
        'SunWM_Avg': 'Irradiance (W/m²)', 'AirTC_Avg': 'Temperature (°C)',
        'RH': 'Humidity (%)', 'WS_ms_S_WVT': 'Wind Speed (m/s)'
    }

    time_series_fig = px.line(df, x='Timestamp', y=variable, 
                             labels={variable: labels[variable]})
    time_series_fig.update_traces(line_color=colors['primary'], 
                                 hovertemplate='%{x|%Y-%m-%d %H:%M}<br>%{y:.2f}')
    time_series_fig.update_layout(
        title=dict(text=f'{labels[variable]} Time Series (2023)', font=dict(size=24)),
        template='plotly_white', font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background'], 
        hovermode='x unified', transition={'duration': 500}
    )
    
    box_fig = px.box(df, x='Month', y=variable, 
                     labels={variable: labels[variable], 'Month': 'Month'})
    box_fig.update_traces(marker_color=colors['accent'], 
                          hovertemplate='Month: %{x}<br>%{y:.2f}')
    box_fig.update_layout(
        title=dict(text=f'{labels[variable]} Distribution by Month (2023)', font=dict(size=24)),
        template='plotly_white', font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background']
    )
    
    df['DayOfYear'] = df['Timestamp'].dt.dayofyear
    temp_pivot = df.pivot_table(values='AirTC_Avg', index='Hour', columns='DayOfYear', aggfunc='mean')
    temp_heatmap = go.Figure(data=go.Heatmap(
        z=temp_pivot.values,
        x=temp_pivot.columns,
        y=temp_pivot.index,
        colorscale='Viridis',
        colorbar=dict(title='Temperature (°C)'),
        hovertemplate='Day: %{x}<br>Hour: %{y}<br>Temp: %{z:.2f}°C<extra></extra>'
    ))
    temp_heatmap.update_layout(
        title=dict(text='Temperature Heatmap (2023)', font=dict(size=24)),
        xaxis_title='Day of Year', yaxis_title='Hour of Day',
        template='plotly_white', font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background']
    )
    
    irr_pivot = df.pivot_table(values='SunWM_Avg', index='Hour', columns='DayOfYear', aggfunc='mean')
    irr_heatmap = go.Figure(data=go.Heatmap(
        z=irr_pivot.values,
        x=irr_pivot.columns,
        y=irr_pivot.index,
        colorscale='Hot',
        colorbar=dict(title='Irradiance (W/m²)'),
        hovertemplate='Day: %{x}<br>Hour: %{y}<br>Irr: %{z:.2f} W/m²<extra></extra>'
    ))
    irr_heatmap.update_layout(
        title=dict(text='Irradiance Heatmap (2023)', font=dict(size=24)),
        xaxis_title='Day of Year', yaxis_title='Hour of Day',
        template='plotly_white', font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background']
    )
    
    scatter_fig = px.scatter(df, x='AirTC_Avg', y='SunWM_Avg', 
                            labels={'AirTC_Avg': 'Temperature (°C)', 'SunWM_Avg': 'Irradiance (W/m²)'},
                            opacity=0.5)
    scatter_fig.update_traces(marker=dict(color=colors['secondary']))
    scatter_fig.update_layout(
        title=dict(text='Temperature vs. Irradiance (2023)', font=dict(size=24)),
        template='plotly_white', font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background']
    )
    
    hist_fig = px.histogram(df, x=variable, nbins=50, 
                           labels={variable: labels[variable]})
    hist_fig.update_traces(marker_color=colors['primary'])
    hist_fig.update_layout(
        title=dict(text=f'{labels[variable]} Distribution (2023)', font=dict(size=24)),
        xaxis_title=labels[variable], yaxis_title='Count',
        template='plotly_white', font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background']
    )
    
    return time_series_fig, box_fig, temp_heatmap, irr_heatmap, scatter_fig, hist_fig

@app.callback(
    [Output('sizing-results', 'children'),
     Output('energy-mix-pie', 'figure'),
     Output('download-sizing', 'data'),
     Output('download-sizing-message', 'children'),
     Output('sizing-store', 'data')],
    [Input('load-offset-slider', 'value'),
     Input('pv-percent-slider', 'value'),
     Input('gen-percent-slider', 'value'),
     Input('battery-daily-energy-slider', 'value'),
     Input('peak-sun-hours', 'value'),
     Input('pv-system-efficiency', 'value'),
     Input('dod', 'value'),
     Input('gen-safety-factor', 'value'),
     Input('download-sizing-btn', 'n_clicks')]
)
def update_sizing(load_offset, pv_percent, gen_percent, battery_daily_energy_percent, peak_sun_hours, 
                  pv_system_efficiency, dod, gen_safety_factor, download_n_clicks):
    if any(x is None for x in [load_offset, pv_percent, gen_percent, battery_daily_energy_percent, 
                               peak_sun_hours, pv_system_efficiency, dod, gen_safety_factor]):
        return (html.P("Please provide all inputs.", style={'color': 'red'}), 
                go.Figure(), None, None, None)

    result, error = size_hybrid_system(load_offset, pv_percent, gen_percent, battery_daily_energy_percent, 
                                       annual_energy_kwh, max_demand_kw, peak_sun_hours, 
                                       pv_system_efficiency, dod, gen_safety_factor)
    
    if error or not result:
        return (html.P(error, style={'color': 'red'}), go.Figure(), None, None, None)

    sizing_table = html.Table([
        html.Thead(html.Tr([
            html.Th("Component"), html.Th("Capacity"), html.Th("Annual Energy Supplied (kWh)"), 
            html.Th("Additional"), html.Th("Cost (R)")
        ])),
        html.Tbody([
            html.Tr([
                html.Td("PV"), 
                html.Td(f"{result['pv_capacity_kwp']:,.2f} kWp, {result['inverter_capacity_kw']:,.2f} kW (inverter)"), 
                html.Td(f"{result['pv_energy_kwh']:,.2f}"), 
                html.Td(f"{result['pv_num_panels']:,} panels (600W)"),
                html.Td(f"R{result['pv_cost']:,.0f}")
            ]),
            html.Tr([
                html.Td("BESS"), 
                html.Td(f"{result['battery_capacity_mwh']:,.3f} MWh, {result['battery_inverter_kw']:,.2f} kW (inverter)"), 
                html.Td("-"), 
                html.Td(f"Daily Energy: {result['battery_daily_energy_kwh']:,.2f} kWh"),
                html.Td(f"R{result['battery_cost']:,.0f}")
            ]),
            html.Tr([
                html.Td("GEN"), 
                html.Td(f"{result['gen_capacity_kw']:,.2f} kW"), 
                html.Td(f"{result['gen_energy_kwh']:,.2f}"), 
                html.Td(f"Daily Energy: {result['gen_capacity_kwh']:,.2f} kWh"),
                html.Td(f"R{result['gen_cost']:,.0f}")
            ]),
            html.Tr([
                html.Td("Total"), 
                html.Td(""), 
                html.Td(""), 
                html.Td(""),
                html.Td(f"R{result['total_cost']:,.0f}")
            ])
        ])
    ], className="table table-bordered table-striped")

    pie_fig = go.Figure(data=[
        go.Pie(
            labels=['PV', 'GEN'],
            values=[result['pv_energy_kwh'], result['gen_energy_kwh']],
            textinfo='percent+label',
            textfont=dict(size=16),
            marker=dict(
                colors=[colors['secondary'], colors['accent']],
                line=dict(color='#ffffff', width=2)
            ),
            hovertemplate='%{label}: %{value:,.2f} kWh<br>%{percent:.1%}<extra></extra>',
            pull=[0.05, 0.05]
        )
    ])
    pie_fig.update_layout(
        title=dict(text='Annual Energy Mix by Source', font=dict(size=24)),
        template='plotly_white',
        font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50, l=50, r=50),
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        showlegend=True,
        legend=dict(orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.5, font=dict(size=16)),
        transition={'duration': 500}
    )

    download_data = None
    download_message = None
    if download_n_clicks > 0:
        sizing_data = pd.DataFrame([result])
        download_data = dcc.send_data_frame(sizing_data.to_csv, 'sizing_results.csv')
        download_message = html.P("Download successful!", style={'color': colors['accent']})
    
    return (sizing_table, pie_fig, download_data, download_message, result)

@app.callback(
    [Output('download-weather', 'data'),
     Output('download-load', 'data'),
     Output('download-message', 'children')],
    [Input('download-weather-btn', 'n_clicks'),
     Input('download-load-btn', 'n_clicks')],
    [State('download-message', 'children')]
)
def download_data(weather_n_clicks, load_n_clicks, current_message):
    ctx = callback_context
    if not ctx.triggered:
        return None, None, current_message

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if button_id == 'download-weather-btn' and weather_n_clicks > 0:
        return (dcc.send_data_frame(df[['Timestamp', 'SunWM_Avg', 'AirTC_Avg', 'RH', 'WS_ms_S_WVT']].to_csv, 
                                   'weather_data.csv'),
                None,
                html.P("Weather data download successful!", style={'color': colors['accent']}))
    
    elif button_id == 'download-load-btn' and load_n_clicks > 0:
        load_data = df[['Timestamp', 'Power_kW']].copy()
        load_data['Energy_kWh'] = load_data['Power_kW'] / 2
        return (None,
                dcc.send_data_frame(load_data.to_csv, 'load_data.csv'),
                html.P("Load data download successful!", style={'color': colors['accent']}))
    
    return None, None, current_message

@app.callback(
    [Output('weekly-source-heatmap', 'figure'),
     Output('monthly-source-heatmap', 'figure'),
     Output('stacked-area', 'figure'),
     Output('source-pie', 'figure'),
     Output('energy-summary-table', 'children')],
    [Input('week-selector', 'value'),
     Input('sizing-store', 'data'),
     Input('reset-week-button', 'n_clicks')],
    prevent_initial_call=False
)
def update_energy_usage(week_index, sizing_data, reset_n_clicks):
    ctx = callback_context
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else None
    
    if trigger_id == 'reset-week-button' and reset_n_clicks > 0:
        week_index = 0

    if not sizing_data or not all(key in sizing_data for key in ['pv_capacity_kwp', 'battery_daily_energy_percent']):
        empty_fig = go.Figure().update_layout(
            title=dict(text='No sizing data available. Please complete sizing in the System Sizing tab.', 
                       font=dict(size=24)), template='plotly_white')
        return (empty_fig, empty_fig, empty_fig, empty_fig, html.P("No sizing data.", style={'color': 'red'}))

    pv_capacity_kwp = sizing_data['pv_capacity_kwp']
    has_battery = sizing_data['battery_daily_energy_percent'] > 0
    try:
        df_with_sources = assign_energy_sources(pv_capacity_kwp, df, has_battery=has_battery)
    except Exception as e:
        empty_fig = go.Figure().update_layout(
            title=dict(text=f'Error processing data: {str(e)}', font=dict(size=24)), template='plotly_white')
        return (empty_fig, empty_fig, empty_fig, empty_fig, html.P(f"Error: {str(e)}", style={'color': 'red'}))

    week_start = pd.to_datetime('2023-01-01') + pd.Timedelta(days=week_index * 7)
    week_end = week_start + pd.Timedelta(days=6, hours=23)
    week_df = df_with_sources[df_with_sources['Timestamp'].between(week_start, week_end)]
    
    if week_df.empty:
        empty_fig = go.Figure().update_layout(
            title=dict(text=f'No data for Week {week_index + 1}', font=dict(size=24)), template='plotly_white')
        return (empty_fig, empty_fig, empty_fig, empty_fig, html.P("No data for selected week.", style={'color': 'red'}))

    heatmap_data = week_df.pivot_table(values='Source_kWh', index='DayOfWeek', columns='Hour', 
                                      aggfunc='mean', fill_value=0)
    source_data = week_df.pivot_table(values='Source', index='DayOfWeek', columns='Hour', 
                                     aggfunc=lambda x: x.mode()[0] if not x.empty else 'None')
    source_map = {'PV': 2, 'BESS': 1, 'GEN': 0, 'None': -1}
    z_data = source_data.apply(lambda x: x.map(source_map.get), axis=1)
    text_data = source_data.apply(lambda x: x.where(x != 'None', ''), axis=1)
    combined_text = text_data
    
    weekly_heatmap_fig = go.Figure()
    weekly_heatmap_fig.add_trace(go.Heatmap(
        z=z_data.values, x=z_data.columns, y=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
        colorscale=[
            [0/3, colors['accent']],
            [1/3, colors['accent']],
            [1/3, colors['primary']],
            [2/3, colors['primary']],
            [2/3, colors['secondary']],
            [3/3, colors['secondary']]
        ],
        zmin=-0.5, zmax=2.5, showscale=False,
        text=combined_text.values, texttemplate='%{text}', textfont=dict(size=16, color='white'),
        xgap=2, ygap=2,
        hovertemplate='Day: %{y}<br>Hour: %{x}<br>Source: %{text}<extra></extra>'
    ))
    for source, color, z_val in [('PV', colors['secondary'], 2), ('BESS', colors['primary'], 1), ('GEN', colors['accent'], 0)]:
        weekly_heatmap_fig.add_trace(go.Scatter(
            x=[None], y=[None], mode='markers', marker=dict(size=10, color=color),
            name=source, showlegend=True
        ))
    weekly_heatmap_fig.update_layout(
        title=dict(text=f'Weekly Energy Source Usage (Week {week_index + 1}: {week_start.strftime("%b %d")}–{week_end.strftime("%b %d")})', 
                   font=dict(size=28), x=0.5, xanchor='center'),
        xaxis_title='Hour of Day', yaxis_title='Day of Week',
        xaxis=dict(tickvals=list(range(0, 24, 3)), ticktext=[str(i) for i in range(0, 24, 3)], 
                   tickfont=dict(size=16), gridcolor='#000000', gridwidth=3),
        yaxis=dict(tickfont=dict(size=16), gridcolor='#000000', gridwidth=3),
        template='plotly_white', font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background'],
        showlegend=True, legend=dict(orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.5)
    )

    monthly_heatmap_data = df_with_sources.pivot_table(values='Source_kWh', index='Month', columns='Hour', 
                                                      aggfunc='mean', fill_value=0)
    monthly_source_data = df_with_sources.pivot_table(values='Source', index='Month', columns='Hour', 
                                                     aggfunc=lambda x: x.mode()[0] if not x.empty else 'None')
    monthly_z_data = monthly_source_data.apply(lambda x: x.map(source_map.get), axis=1)
    monthly_text_data = monthly_source_data.apply(lambda x: x.where(x != 'None', ''), axis=1)
    monthly_combined_text = monthly_text_data
    
    monthly_heatmap_fig = go.Figure()
    monthly_heatmap_fig.add_trace(go.Heatmap(
        z=monthly_z_data.values, x=monthly_z_data.columns, y=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                                              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        colorscale=[
            [0/3, colors['accent']],
            [1/3, colors['accent']],
            [1/3, colors['primary']],
            [2/3, colors['primary']],
            [2/3, colors['secondary']],
            [3/3, colors['secondary']]
        ],
        zmin=-0.5, zmax=2.5, showscale=False,
        text=monthly_combined_text.values, texttemplate='%{text}', textfont=dict(size=16, color='white'),
        xgap=2, ygap=2,
        hovertemplate='Month: %{y}<br>Hour: %{x}<br>Source: %{text}<extra></extra>'
    ))
    for source, color, z_val in [('PV', colors['secondary'], 2), ('BESS', colors['primary'], 1), ('GEN', colors['accent'], 0)]:
        monthly_heatmap_fig.add_trace(go.Scatter(
            x=[None], y=[None], mode='markers', marker=dict(size=10, color=color),
            name=source, showlegend=True
        ))
    monthly_heatmap_fig.update_layout(
        title=dict(text='Monthly Energy Source Usage (2023)', font=dict(size=28), x=0.5, xanchor='center'),
        xaxis_title='Hour of Day', yaxis_title='Month',
        xaxis=dict(tickvals=list(range(0, 24, 3)), ticktext=[str(i) for i in range(0, 24, 3)], 
                   tickfont=dict(size=16), gridcolor='#000000', gridwidth=3),
        yaxis=dict(tickfont=dict(size=16), gridcolor='#000000', gridwidth=3),
        template='plotly_white', font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background'],
        showlegend=True, legend=dict(orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.5)
    )

    week_df = week_df.copy()
    week_df['PV_kWh'] = week_df.apply(lambda x: x['Source_kWh'] if x['Source'] == 'PV' else 0, axis=1)
    week_df['BESS_kWh'] = week_df.apply(lambda x: x['Source_kWh'] if x['Source'] == 'BESS' else 0, axis=1)
    week_df['GEN_kWh'] = week_df.apply(lambda x: x['Source_kWh'] if x['Source'] == 'GEN' else 0, axis=1)
    week_df['Load_kWh'] = week_df['Power_kW'] / 2
    area_fig = go.Figure(data=[
        go.Scatter(x=week_df['Timestamp'], y=week_df['PV_kWh'], 
                   mode='none', stackgroup='one', name='PV', fillcolor=colors['secondary'],
                   hovertemplate='PV: %{y:.2f} kWh<extra></extra>'),
        go.Scatter(x=week_df['Timestamp'], y=week_df['BESS_kWh'], 
                   mode='none', stackgroup='one', name='BESS', fillcolor=colors['primary'],
                   hovertemplate='BESS: %{y:.2f} kWh<extra></extra>'),
        go.Scatter(x=week_df['Timestamp'], y=week_df['GEN_kWh'], 
                   mode='none', stackgroup='one', name='GEN', fillcolor=colors['accent'],
                   hovertemplate='GEN: %{y:.2f} kWh<extra></extra>'),
        go.Scatter(x=week_df['Timestamp'], y=week_df['Load_kWh'], 
                   mode='lines', name='Load', line=dict(color=colors['load'], dash='dash', width=2),
                   hovertemplate='Load: %{y:.2f} kWh<extra></extra>')
    ])
    area_fig.update_layout(
        title=dict(text=f'Energy Contribution by Source with Load (Week {week_index + 1}: {week_start.strftime("%b %d")}–{week_end.strftime("%b %d")})', 
                   font=dict(size=24), x=0.5, xanchor='center'),
        xaxis_title='Date', yaxis_title='Energy (kWh)',
        xaxis=dict(tickfont=dict(size=16)), yaxis=dict(tickfont=dict(size=16)),
        template='plotly_white', font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background'], 
        hovermode='x unified'
    )

    pv_total = week_df[week_df['Source'] == 'PV']['Source_kWh'].sum()
    bess_total = week_df[week_df['Source'] == 'BESS']['Source_kWh'].sum()
    gen_total = week_df[week_df['Source'] == 'GEN']['Source_kWh'].sum()
    pie_fig = go.Figure(data=[
        go.Pie(
            labels=['PV', 'BESS', 'GEN'],
            values=[pv_total, bess_total, gen_total],
            textinfo='percent+label',
            textfont=dict(size=16),
            marker=dict(
                colors=[colors['secondary'], colors['primary'], colors['accent']],
                line=dict(color='#ffffff', width=2)
            ),
            hovertemplate='%{label}: %{value:,.2f} kWh<br>%{percent:.1%}<extra></extra>',
            pull=[0.05, 0.05, 0.05]
        )
    ])
    pie_fig.update_layout(
        title=dict(text=f'Energy Source Distribution (Week {week_index + 1}: {week_start.strftime("%b %d")}–{week_end.strftime("%b %d")})', 
                   font=dict(size=24), x=0.5, xanchor='center'),
        template='plotly_white',
        font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background'],
        showlegend=True,
        legend=dict(orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.5, font=dict(size=16))
    )

    load_total = week_df['Load_kWh'].sum()
    summary_table = html.Table([
        html.Thead(html.Tr([html.Th("Source"), html.Th("Energy Supplied (kWh)")])),
        html.Tbody([
            html.Tr([html.Td("PV"), html.Td(f"{pv_total:,.2f}")]),
            html.Tr([html.Td("BESS"), html.Td(f"{bess_total:,.2f}")]),
            html.Tr([html.Td("GEN"), html.Td(f"{gen_total:,.2f}")]),
            html.Tr([html.Td("Load"), html.Td(f"{load_total:,.2f}")])
        ])
    ], className="table table-bordered table-striped", style={'font-size': '16px', 'margin': 'auto', 'width': '50%'})

    return (weekly_heatmap_fig, monthly_heatmap_fig, area_fig, pie_fig, summary_table)

@app.callback(
    [Output('financial-metrics-table', 'children'),
     Output('cost-breakdown-pie', 'figure'),
     Output('payback-period-chart', 'figure'),
     Output('lcoe-gauge', 'figure'),
     Output('co2-impact-chart', 'figure'),
     Output('co2-savings-text', 'children')],
    [Input('pv-cost-input', 'value'),
     Input('bess-cost-input', 'value'),
     Input('gen-cost-input', 'value'),
     Input('fuel-type', 'value'),
     Input('fuel-cost-input', 'value'),
     Input('discount-rate', 'value'),
     Input('project-lifetime', 'value'),
     Input('electricity-cost', 'value'),
     Input('pv-om', 'value'),
     Input('bess-om', 'value'),
     Input('gen-om', 'value'),
     Input('sizing-store', 'data')]
)
def update_financials(pv_cost_per_kwp, bess_cost_per_kwh, gen_cost_per_kw, fuel_type, fuel_cost, 
                      discount_rate, project_lifetime, electricity_cost, pv_om, bess_om, gen_om, sizing_data):
    if not sizing_data or any(x is None for x in [pv_cost_per_kwp, bess_cost_per_kwh, gen_cost_per_kw, 
                                                  fuel_type, fuel_cost, discount_rate, project_lifetime, 
                                                  electricity_cost, pv_om, bess_om, gen_om]):
        return (html.P("Please complete sizing and provide all inputs.", style={'color': 'red'}),
                go.Figure(), go.Figure(), go.Figure(), go.Figure(), "")

    metrics = calculate_financial_metrics(
        sizing_data, pv_cost_per_kwp, bess_cost_per_kwh, gen_cost_per_kw, 
        discount_rate, project_lifetime, electricity_cost, 
        pv_om, bess_om, gen_om, fuel_type, fuel_cost
    )

    if not metrics:
        return (html.P(metrics[1], style={'color': 'red'}),
                go.Figure(), go.Figure(), go.Figure(), go.Figure(), "")

    metrics_table = html.Table([
        html.Thead(html.Tr([html.Th("Metric"), html.Th("Value")])),
        html.Tbody([
            html.Tr([html.Td("Total Capital Cost"), html.Td(f"R{metrics['total_capital_cost']:,.0f}")]),
            html.Tr([html.Td("Net Present Value (NPV)"), html.Td(f"R{metrics['npv']:,.0f}")]),
            html.Tr([html.Td("Internal Rate of Return (IRR)"), html.Td(f"{metrics['irr']:.2f}%")]),
            html.Tr([html.Td("Payback Period"), html.Td(f"{metrics['payback_period']:.2f} years" if metrics['payback_period'] != float('inf') else "N/A")]),
            html.Tr([html.Td("Levelized Cost of Energy (LCOE)"), html.Td(f"R{metrics['lcoe']:.4f}/kWh")]),
            html.Tr([html.Td("Annual Savings"), html.Td(f"R{metrics['annual_savings']:,.0f}")]),
            html.Tr([html.Td("Annual O&M Cost"), html.Td(f"R{metrics['total_om_cost']:,.0f}")]),
            html.Tr([html.Td("Annual Fuel Cost"), html.Td(f"R{metrics['annual_fuel_cost']:,.0f}")])
        ])
    ], className="table table-bordered table-striped", style={'font-size': '16px', 'margin': 'auto', 'width': '80%'})

    cost_pie_fig = go.Figure(data=[
        go.Pie(
            labels=['PV', 'BESS', 'GEN'],
            values=[metrics['pv_cost'], metrics['bess_cost'], metrics['gen_cost']],
            textinfo='percent+label',
            textfont=dict(size=16),
            marker=dict(
                colors=[colors['secondary'], colors['primary'], colors['accent']],
                line=dict(color='#ffffff', width=2)
            ),
            hovertemplate='%{label}: R%{value:,.0f}<br>%{percent:.1%}<extra></extra>',
            pull=[0.05, 0.05, 0.05]
        )
    ])
    cost_pie_fig.update_layout(
        title=dict(text='Capital Cost Breakdown', font=dict(size=24), x=0.5, xanchor='center'),
        template='plotly_white',
        font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50, l=50, r=50),
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        showlegend=True,
        legend=dict(orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.5, font=dict(size=16)),
        transition={'duration': 500}
    )

    cumulative_cash_flows = [sum(metrics['cash_flows'][:i+1]) for i in range(len(metrics['cash_flows']))]
    years = list(range(len(metrics['cash_flows'])))
    payback_fig = go.Figure()
    payback_fig.add_trace(go.Scatter(
        x=years, y=cumulative_cash_flows, mode='lines+markers', name='Cumulative Cash Flow',
        line=dict(color=colors['primary'], width=2),
        marker=dict(size=8),
        hovertemplate='Year: %{x}<br>Cumulative Cash Flow: R%{y:,.0f}<extra></extra>'
    ))
    payback_fig.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="Break-even", 
                          annotation_position="top left")
    if metrics['payback_period'] != float('inf'):
        payback_fig.add_vline(x=metrics['payback_period'], line_dash="dash", line_color=colors['accent'],
                              annotation_text=f"Payback: {metrics['payback_period']:.2f} years", 
                              annotation_position="top right")
    payback_fig.update_layout(
        title=dict(text='Cumulative Cash Flow and Payback Period', font=dict(size=24), x=0.5, xanchor='center'),
        xaxis_title='Year', yaxis_title='Cumulative Cash Flow (R)',
        xaxis=dict(tickfont=dict(size=16)), yaxis=dict(tickfont=dict(size=16)),
        template='plotly_white', font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50), plot_bgcolor=colors['background'], 
        paper_bgcolor=colors['background'],
        hovermode='x unified'
    )

    lcoe = metrics['lcoe']
    lcoe_gauge_fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=lcoe,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Levelized Cost of Energy (R/kWh)", 'font': {'size': 24}},
        gauge={
            'axis': {'range': [0, 3], 'tickwidth': 1, 'tickcolor': colors['text']},
            'bar': {'color': colors['primary']},
            'steps': [
                {'range': [0, 0.5], 'color': '#90EE90'},
                {'range': [0.5, 1.5], 'color': '#FFD700'},
                {'range': [1.5, 3], 'color': '#FF4500'}
            ],
            'threshold': {
                'line': {'color': colors['accent'], 'width': 4},
                'thickness': 0.75,
                'value': 1.5
            }
        }
    ))
    lcoe_gauge_fig.update_layout(
        template='plotly_white',
        font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50, l=50, r=50),
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background']
    )

    co2_fig = go.Figure(data=[
        go.Bar(
            x=['PV CO2 Savings', 'Generator CO2 Emissions'],
            y=[metrics['co2_savings_pv']/1000, metrics['co2_emissions_gen']/1000],
            marker_color=[colors['secondary'], colors['accent']],
            text=[f"{metrics['co2_savings_pv']/1000:,.2f}", f"{metrics['co2_emissions_gen']/1000:,.2f}"],
            textposition='auto',
            hovertemplate='%{x}: %{y:,.2f} tonnes CO2<extra></extra>'
        )
    ])
    co2_fig.update_layout(
        title=dict(text='CO2 Impact (Annual)', font=dict(size=24), x=0.5, xanchor='center'),
        yaxis_title='CO2 (Tonnes)',
        template='plotly_white',
        font=dict(family='Arial', size=18, color=colors['text']),
        margin=dict(t=50, b=50),
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        showlegend=False
    )

    co2_savings_text = f"PV CO2 Savings: {metrics['co2_savings_pv']/1000:,.2f} tonnes/year, equivalent to planting {int(metrics['trees_planted']):,} trees annually."

    return (metrics_table, cost_pie_fig, payback_fig, lcoe_gauge_fig, co2_fig, co2_savings_text)

@app.callback(
    [Output('prediction-plot', 'figure'),
     Output('prediction-message', 'children')],
    [Input('pred-week-selector', 'value'),
     Input('sizing-store', 'data')]
)
def update_prediction_plot(week_index, sizing_data):
    if not sizing_data or 'pv_capacity_kwp' not in sizing_data:
        return (go.Figure().update_layout(
                    title=dict(text='No sizing data available. Please complete sizing in the System Sizing tab.', 
                               font=dict(size=24, family='Arial', color=colors['text']), x=0.5, xanchor='center'),
                    template='plotly_white'),
                html.P("No sizing data.", style={'color': 'red', 'font-family': 'Arial', 'font-size': '16px'}))

    pv_capacity_kwp = sizing_data['pv_capacity_kwp']
    week_start = pd.to_datetime('2023-01-01') + pd.Timedelta(days=week_index * 7)
    week_end = week_start + pd.Timedelta(days=6, hours=23)
    
    pred_pv, pred_load, timestamps, error = train_predict_pv_load(df, pv_capacity_kwp, week_start, week_end)
    
    if error:
        return (go.Figure().update_layout(
                    title=dict(text=error, font=dict(size=24, family='Arial', color=colors['text']), x=0.5, xanchor='center'),
                    template='plotly_white'),
                html.P(error, style={'color': 'red', 'font-family': 'Arial', 'font-size': '16px'}))

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=timestamps, 
        y=pred_pv, 
        mode='lines', 
        name='Predicted PV Generation',
        line=dict(color=colors['secondary'], width=3),
        hovertemplate='%{x|%Y-%m-%d %H:%M}<br>PV: %{y:.2f} kW<extra></extra>'
    ))
    fig.add_trace(go.Scatter(
        x=timestamps, 
        y=pred_load, 
        mode='lines', 
        name='Predicted Load',
        line=dict(color=colors['load'], width=3, dash='dash'),
        hovertemplate='%{x|%Y-%m-%d %H:%M}<br>Load: %{y:.2f} kW<extra></extra>'
    ))
    fig.update_layout(
        title=dict(
            text=f'Predicted PV Generation and Load (Week {week_index + 1}: {week_start.strftime("%b %d")}–{week_end.strftime("%b %d")})',
            font=dict(size=28, family='Arial', color=colors['text'], weight='bold'),
            x=0.5, 
            xanchor='center'
        ),
        xaxis_title='Date',
        yaxis_title='Power (kW)',
        xaxis=dict(
            tickfont=dict(size=14, family='Arial', color=colors['text']),
            gridcolor='#e0e0e0',
            title_font=dict(size=18, family='Arial', color=colors['text'])
        ),
        yaxis=dict(
            tickfont=dict(size=14, family='Arial', color=colors['text']),
            gridcolor='#e0e0e0',
            title_font=dict(size=18, family='Arial', color=colors['text'])
        ),
        template='plotly_white',
        font=dict(family='Arial', size=16, color=colors['text']),
        margin=dict(t=80, b=100, l=60, r=60),
        plot_bgcolor='#ffffff',
        paper_bgcolor='#ffffff',
        hovermode='x unified',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.3,
            xanchor='center',
            x=0.5,
            font=dict(size=14, family='Arial', color=colors['text']),
            bgcolor='#ffffff',
            bordercolor=colors['text'],
            borderwidth=1
        ),
        showlegend=True
    )

    return fig, html.P(f"Predictions for Week {week_index + 1} based on XGBoost model.", 
                       style={'color': colors['accent'], 'font-family': 'Arial', 'font-size': '16px'})

@app.callback(
    Output('download-pdf', 'data'),
    Input('download-pdf-btn', 'n_clicks'),
    prevent_initial_call=True
)
def download_pdf(n_clicks):
    if n_clicks > 0:
        html_content = """
        <html>
        <head>
            <title>Hybrid Energy System Dashboard</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }
                h1, h3, h4, h5 { text-align: center; color: #333333; }
                .section { margin-bottom: 40px; padding: 20px; background-color: #ffffff; border-radius: 10px; }
                .table { width: 80%; margin: auto; border-collapse: collapse; }
                .table th, .table td { border: 1px solid #333333; padding: 8px; text-align: center; }
                img { display: block; margin: 20px auto; max-width: 100%; }
                p { text-align: center; font-size: 16px; color: #333333; }
            </style>
        </head>
        <body>
            <div class="section">
                <h1>Hybrid Energy System Dashboard</h1>
                <h3>Project Information</h3>
                <img src="assets/Logo.jpg" alt="Stellenbosch University Logo" style="height: 150px; border-radius: 10px;">
                <h4>Stellenbosch University</h4>
                <h5>Faculty of Engineering</h5>
                <h5>Department of Electrical & Electronics Engineering</h5>
                <h5>Power System Data Analytics 874 (Final Project)</h5>
                <p>This dashboard facilitates the design and analysis of off-grid hybrid energy systems. It enables users to size photovoltaic (PV), battery, and generator components to meet a specified load offset, visualize energy usage patterns through interactive heatmaps and charts, and explore weather data impacting system performance. Developed for industrial applications, the tool provides dynamic, data-driven insights for sustainable energy planning.</p>
                <h5>Panashe Bradley Kapfunde (27692566)</h5>
                <h5>Title: A Data Visualization Dashboard for Sizing Battery and PV Systems and Predicting Energy Output and Savings for Off-Grid Energy Systems</h5>
            </div>
            <div class="section">
                <h3>Weather Data Visualization</h3>
                <p>Visualizations include time series, box plots, heatmaps, scatter plots, and histograms of weather variables (Irradiance, Temperature, Humidity, Wind Speed) for 2023.</p>
            </div>
            <div class="section">
                <h3>System Sizing</h3>
                <p>Interactive sliders and inputs allow users to size PV, BESS, and GEN components based on load offset, PV/GEN percentages, and other parameters. Results include capacities, energy supplied, and costs.</p>
            </div>
            <div class="section">
                <h3>Energy Usage</h3>
                <p>Displays weekly and monthly energy source usage (PV, BESS, GEN) via heatmaps, stacked area charts, and pie charts, with a summary table for the selected week.</p>
            </div>
            <div class="section">
                <h3>Financial Analysis and CO2 Impact</h3>
                <p>Provides financial metrics (NPV, IRR, Payback Period, LCOE), cost breakdowns, and CO2 savings/emissions for the hybrid system.</p>
            </div>
            <div class="section">
                <h3>PV and Load Predictions</h3>
                <p>Predicts PV generation and load demand for a selected week in 2023 using an XGBoost model, displayed as a line plot.</p>
            </div>
        </body>
        </html>
        """
        with open('dashboard.html', 'w') as f:
            f.write(html_content)
        return dcc.send_file('dashboard.html', filename='Hybrid_Energy_Dashboard.html')

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

Maximum Power_kW after cleaning: 2041.00 kW
Number of zero values in Power_kW: 0


Exception: Login Required